# Goal
This is to use one of the previous training/inference pipeline and tie it to evlidently

In [1]:
from evidently.ui.workspace import RemoteWorkspace

### Create Project

In [2]:
ws = RemoteWorkspace("http://localhost:8000")
project = ws.create_project("Test")
project.description = "My project description"
project.save()

In [ ]:
import pandas as pd
from sklearn import datasets
    
from evidently import Dataset
from evidently import DataDefinition
from evidently import Report
from evidently.presets import DataDriftPreset, DataSummaryPreset


### Get data & create report

In [ ]:
adult_data = datasets.fetch_openml(name="adult", version=2, as_frame="auto")
adult = adult_data.frame
adult_ref = adult[~adult.education.isin(["Some-college", "HS-grad", "Bachelors"])]
adult_prod = adult[adult.education.isin(["Some-college", "HS-grad", "Bachelors"])]
schema = DataDefinition(
    numerical_columns=["education-num", "age", "capital-gain", "hours-per-week", "capital-loss", "fnlwgt"],
    categorical_columns=["education", "occupation", "native-country", "workclass", "marital-status", "relationship", "race", "sex", "class"],
    )
eval_data_1 = Dataset.from_pandas(
    pd.DataFrame(adult_prod),
    data_definition=schema
)
eval_data_2 = Dataset.from_pandas(
    pd.DataFrame(adult_ref),
    data_definition=schema
)
report = Report([
    DataDriftPreset() 
])
my_eval = report.run(eval_data_1, eval_data_2)


In [10]:
ws.add_run(project.id, my_eval, include_data=False)

Report ID: 019741af-219d-76ad-aafc-0c81099820ad
Link: http://localhost:8000/projects/019741aa-c333-7471-a129-de97c15d6e2a/reports/019741af-219d-76ad-aafc-0c81099820ad

### Data analysis

In [11]:
df = pd.DataFrame(adult_prod)
df.describe(include="all")

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
count,34687.000000,32784,3.468700e+04,34687,34687.000000,34687,32780,34687,34687,34687,34687.000000,34687.000000,34687.000000,34124,34687
unique,NaN,8,NaN,3,NaN,7,14,6,5,2,NaN,NaN,NaN,41,2
top,NaN,Private,NaN,HS-grad,NaN,Married-civ-spouse,Adm-clerical,Husband,White,Male,NaN,NaN,NaN,United-States,<=50K
freq,NaN,24520,NaN,15784,NaN,15558,4670,13682,29710,22935,NaN,NaN,NaN,31848,26808
mean,37.967509,NaN,1.889974e+05,NaN,10.239023,NaN,NaN,NaN,NaN,NaN,844.265489,83.323925,40.510220,NaN,NaN
std,13.308068,NaN,1.053749e+05,NaN,1.574856,NaN,NaN,NaN,NaN,NaN,6159.971515,391.262436,11.944506,NaN,NaN
min,17.000000,NaN,1.228500e+04,NaN,9.000000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,1.000000,NaN,NaN
25%,27.000000,NaN,1.176180e+05,NaN,9.000000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,40.000000,NaN,NaN
50%,36.000000,NaN,1.776750e+05,NaN,10.000000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,40.000000,NaN,NaN
75%,47.000000,NaN,2.363935e+05,NaN,10.000000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,45.000000,NaN,NaN


In [19]:


my_eval.json()


'{"metrics": [{"id": "15e89f895b482f9b84ba7274ed18a106", "metric_id": "DriftedColumnsCount(drift_share=0.5)", "value": {"count": 5.0, "share": 0.3333333333333333}}, {"id": "84c249c384495467eda04c510443557a", "metric_id": "ValueDrift(column=education-num)", "value": 0.6176968555693167}, {"id": "1cf7e5c2a8eb2a62a2667e70ee06d259", "metric_id": "ValueDrift(column=age)", "value": 0.18534692319042428}, {"id": "5cf10e0f4a964f3f2c5fbeb480939633", "metric_id": "ValueDrift(column=capital-gain)", "value": 0.0817732650223179}, {"id": "2e8359ba1142f9250d66ab64b4bde735", "metric_id": "ValueDrift(column=hours-per-week)", "value": 0.08859914569715911}, {"id": "732ea745dc73e93174c7cab970ff7513", "metric_id": "ValueDrift(column=capital-loss)", "value": 0.03378837284314874}, {"id": "e4e7c5caf091394a8fba77dca1af3931", "metric_id": "ValueDrift(column=fnlwgt)", "value": 0.02364332069106699}, {"id": "3a4389f62f6a9720415e27025a4f8bd6", "metric_id": "ValueDrift(column=education)", "value": 0.8325546111576977},